In [ ]:
TRAIN_CSV_PATH='/kaggle/input/ksu-july-dataset/train_feat.csv'
TEST_CSV_PATH='/kaggle/input/ksu-july-dataset/test_feat.csv'



In [1]:
import numpy as np

def median_absolute_percentage_error(y_true, y_pred, *, sample_weight=None, multioutput="uniform_average"):
    """
    Median absolute percentage error regression loss.
    Compatible with sklearn-style usage.
    """
    y_true = np.array(y_true, dtype=np.float64).ravel()
    y_pred = np.array(y_pred, dtype=np.float64).ravel()

    # avoid division by zero
    nonzero_mask = y_true != 0
    if not np.any(nonzero_mask):
        raise ValueError("All y_true values are zero; MAPE cannot be calculated.")

    errors = np.abs((y_true[nonzero_mask] - y_pred[nonzero_mask]) / y_true[nonzero_mask])

    return np.median(errors)


In [2]:
import random
import joblib  
from itertools import product
from pathlib import Path

import numpy as np
import pandas as pd

import shap
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error,r2_score,mean_absolute_percentage_error
from tqdm import tqdm

import optuna
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler


# ====== Storage for plotting ======
history = []

# ====== Objective ======


def xgb_objective(trial):
    # Extensive hyperparameter space
    params = {
        "objective": "reg:squarederror",
        "eval_metric": "mae",
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        
        # Tree structure parameters
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 15),
        "gamma": trial.suggest_float("gamma", 0, 10, step=0.5),
        
        # Learning rate and estimators
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 300, 3000, step=100),
        
        # Subsampling parameters
        "subsample": trial.suggest_float("subsample", 0.5, 1.0, step=0.05),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0, step=0.05),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0, step=0.05),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0, step=0.05),
        
        # Regularization parameters
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        
        # DART-specific parameters
        "sample_type": trial.suggest_categorical("sample_type", ["uniform", "weighted"]),
        "normalize_type": trial.suggest_categorical("normalize_type", ["tree", "forest"]),
        "rate_drop": trial.suggest_float("rate_drop", 0.0, 0.5, step=0.05),
        "skip_drop": trial.suggest_float("skip_drop", 0.0, 0.9, step=0.1),
    }
    
    # Data prep
    y_train_np = y_train.values
    y_val_np = y_val.values
    X_train_np = X_train.values
    X_val_np = X_val.values

    scaler = StandardScaler()
    X_train_np = scaler.fit_transform(X_train_np)
    X_val_np = scaler.transform(X_val_np)
    
    # Model training
    model = XGBRegressor(**params, random_state=42, n_jobs=-1)
    model.fit(
        X_train_np, y_train_np,
        eval_set=[(X_val_np, y_val_np)],
        early_stopping_rounds=100,
        verbose=False
    )
    
    # Validation MAE
    y_val_pred = model.predict(X_val_np)
    val_mae = mean_absolute_error(y_val_np, y_val_pred)
    
    # Training MAE
    y_train_pred = model.predict(X_train_np)
    train_mae = mean_absolute_error(y_train_np, y_train_pred)
    
    # Log results
    history.append({
        **params,
        "val_mae": val_mae,
        "train_mae": train_mae
    })
    
    return val_mae

# ====== Objective ======
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

def rf_objective(trial):
    # Extensive hyperparameter search space
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 2, 100),  # deeper search
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 50),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "min_weight_fraction_leaf": trial.suggest_float("min_weight_fraction_leaf", 0.0, 0.5, step=0.05),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None, 0.2, 0.5, 0.8]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True]),
        "max_samples": trial.suggest_float("max_samples", 0.3, 1.0, step=0.05),
        "oob_score": trial.suggest_categorical("oob_score", [True, False]),
        "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 0.0, 1.0, step=0.01),
        "ccp_alpha": trial.suggest_float("ccp_alpha", 0.0, 0.05, step=0.001)  # complexity pruning
    }

    # Prepare data
    y_train_np = y_train.values
    y_val_np = y_val.values
    X_train_np = X_train.values
    X_val_np = X_val.values

    scaler = StandardScaler()
    X_train_np = scaler.fit_transform(X_train_np)
    X_val_np = scaler.transform(X_val_np)
    
    # Train model
    rf_model = RandomForestRegressor(
        **{k: v for k, v in params.items() if v is not None},
        random_state=42, n_jobs=-1
    )
    rf_model.fit(X_train_np, y_train_np)
    
    # Predictions
    y_val_pred = rf_model.predict(X_val_np)
    val_mae = mean_absolute_error(y_val_np, y_val_pred)
    
    y_train_pred = rf_model.predict(X_train_np)
    train_mae = mean_absolute_error(y_train_np, y_train_pred)
    
    # Log results
    history.append({
        **params,
        "val_mae": val_mae,
        "train_mae": train_mae
    })
    
    return val_mae


from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

def etr_objective(trial):
    # Extensive hyperparameter search space
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 2, 100),  # allow shallow to very deep
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 50),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "min_weight_fraction_leaf": trial.suggest_float("min_weight_fraction_leaf", 0.0, 0.5, step=0.05),
        "max_features": trial.suggest_categorical(
            "max_features", ["sqrt", "log2", None, 0.2, 0.5, 0.8]
        ),
        "bootstrap": trial.suggest_categorical("bootstrap", [True]),
        "max_samples": trial.suggest_float("max_samples", 0.3, 1.0, step=0.05)
                       if trial.suggest_categorical("bootstrap_for_max_samples", [True, False]) else None,
        "min_impurity_decrease": trial.suggest_float("min_impurity_decrease", 0.0, 1.0, step=0.01),
        "ccp_alpha": trial.suggest_float("ccp_alpha", 0.0, 0.05, step=0.001)
    }
    params["oob_score"] = params["bootstrap"]

    # Prepare data
    y_train_np = y_train.values
    y_val_np = y_val.values
    X_train_np = X_train.values
    X_val_np = X_val.values

    scaler = StandardScaler()
    X_train_np = scaler.fit_transform(X_train_np)
    X_val_np = scaler.transform(X_val_np)
    
    # Train model
    etr_model = ExtraTreesRegressor(
        **{k: v for k, v in params.items() if v is not None},
        random_state=42, n_jobs=-1
    )
    etr_model.fit(X_train_np, y_train_np)
    
    # Predictions
    y_val_pred = etr_model.predict(X_val_np)
    val_mae = mean_absolute_error(y_val_np, y_val_pred)
    
    y_train_pred = etr_model.predict(X_train_np)
    train_mae = mean_absolute_error(y_train_np, y_train_pred)
    
    # Log results
    history.append({
        **params,
        "val_mae": val_mae,
        "train_mae": train_mae
    })
    
    return val_mae




In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
import torch
import numpy as np

def score_generator(X, y, model, res_type, device="cpu", is_torch=False):
    # Standardize using training set statistics
    scaler = StandardScaler()
    scaler.fit(X_train)  # Fit only on training data

    y_np = y.values.reshape(-1, 1)
    X_np = X.values
    X_np = scaler.transform(X_np)

    y_pred = model.predict(X_np)  # already NumPy

    # Metrics
    mae = mean_absolute_error(y_np, y_pred)
    mape = median_absolute_percentage_error(y_np, y_pred)
    r2 = r2_score(y_np, y_pred)

    # Print results
    print(f"{res_type}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Median Absolute Percentage Error (MAPE): {mape:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print("=" * 25)

In [4]:
import numpy as np
def featurizer(train_df,test_df):
    # Compute derived features for alloy analysis
    train_df["W/L"]=train_df["Width (mm)"]/train_df["Length (mm)"]
    train_df["L/T"]=train_df["Length (mm)"]/train_df["Thickness (mm)"]
    train_df["W/T"]=train_df["Thickness (mm)"]/train_df["Width (mm)"]
    train_df["Total_Alloy_wt"] = train_df[[
        "C, wt.%", "Si, wt.%", "Mn, wt.%", "P, wt.%", "S, wt.%",
        "Ni, wt.%", "Cr, wt.%", "Mo, wt.%", "Al, wt.%", "N, wt.%",
        "Ti, wt.%", "Nb, wt.%", "B, wt.%", "Cu, wt.%", "V, wt.%",
        "Zr, wt.%", "W, wt.%", "Sn, wt.%", "Co, wt.%", "O, wt,%",
        "Ta, wt.%", "As, wt%", "Sb, wt.%"
    ]].sum(axis=1)

    train_df["Embrittling_Elements"] = (
        train_df["P, wt.%"] +
        train_df["S, wt.%"] +
        train_df["Cu, wt.%"] +
        train_df["As, wt%"] +
        train_df["Sb, wt.%"] +
        train_df["B, wt.%"]
    )

    train_df["Toughness_Enhancers"] = (
        train_df["Ni, wt.%"] +
        train_df["Mn, wt.%"] +
        train_df["Mo, wt.%"] +
        train_df["V, wt.%"] +
        train_df["Ti, wt.%"] +
        train_df["Nb, wt.%"]
    )

    train_df["Hardener_Elements"] = (
        train_df["C, wt.%"] +
        train_df["Cr, wt.%"] +
        train_df["Mo, wt.%"] +
        train_df["V, wt.%"] +
        train_df["W, wt.%"]
    )

    train_df["Aging_Sensitive"] = (
        train_df["Cu, wt.%"] +
        train_df["Ni, wt.%"] +
        train_df["P, wt.%"]
    )

    train_df["Carbon_Equivalent"] = (
        train_df["C, wt.%"] +
        (train_df["Mn, wt.%"] + train_df["Si, wt.%"]) / 6 +
        (train_df["Cr, wt.%"] + train_df["Mo, wt.%"] + train_df["V, wt.%"]) / 5 +
        (train_df["Ni, wt.%"] + train_df["Cu, wt.%"]) / 15
    )

    train_df["Fe_Deficit"] = 100 - train_df["Fe, wt.%"]

    # Compute derived features for alloy analysis
    test_df["W/L"]=test_df["Width (mm)"]/test_df["Length (mm)"]
    test_df["L/T"]=test_df["Length (mm)"]/test_df["Thickness (mm)"]
    test_df["W/T"]=test_df["Thickness (mm)"]/test_df["Width (mm)"]
    test_df["Total_Alloy_wt"] = test_df[[
        "C, wt.%", "Si, wt.%", "Mn, wt.%", "P, wt.%", "S, wt.%",
        "Ni, wt.%", "Cr, wt.%", "Mo, wt.%", "Al, wt.%", "N, wt.%",
        "Ti, wt.%", "Nb, wt.%", "B, wt.%", "Cu, wt.%", "V, wt.%",
        "Zr, wt.%", "W, wt.%", "Sn, wt.%", "Co, wt.%", "O, wt,%",
        "Ta, wt.%", "As, wt%", "Sb, wt.%"
    ]].sum(axis=1)
    test_df["Embrittling_Elements"] = (
        test_df["P, wt.%"] +
        test_df["S, wt.%"] +
        test_df["Cu, wt.%"] +
        test_df["As, wt%"] +
        test_df["Sb, wt.%"] +
        test_df["B, wt.%"]
    )

    test_df["Toughness_Enhancers"] = (
        test_df["Ni, wt.%"] +
        test_df["Mn, wt.%"] +
        test_df["Mo, wt.%"] +
        test_df["V, wt.%"] +
        test_df["Ti, wt.%"] +
        test_df["Nb, wt.%"]
    )

    test_df["Hardener_Elements"] = (
        test_df["C, wt.%"] +
        test_df["Cr, wt.%"] +
        test_df["Mo, wt.%"] +
        test_df["V, wt.%"] +
        test_df["W, wt.%"]
    )

    test_df["Aging_Sensitive"] = (
        test_df["Cu, wt.%"] +
        test_df["Ni, wt.%"] +
        test_df["P, wt.%"]
    )

    test_df["Carbon_Equivalent"] = (
        test_df["C, wt.%"] +
        (test_df["Mn, wt.%"] + test_df["Si, wt.%"]) / 6 +
        (test_df["Cr, wt.%"] + test_df["Mo, wt.%"] + test_df["V, wt.%"]) / 5 +
        (test_df["Ni, wt.%"] + test_df["Cu, wt.%"]) / 15
    )

    test_df["Fe_Deficit"] = 100 - test_df["Fe, wt.%"]

    eps = 1e-6  # small value to avoid divide-by-zero


    # 1. Composition Ratios
    train_df['C_Mn'] = train_df['C, wt.%'] / (train_df['Mn, wt.%'] + eps)
    train_df['Cr_Mo_V_Ni'] = train_df[['Cr, wt.%', 'Mo, wt.%', 'V, wt.%', 'Ni, wt.%']].sum(axis=1)
    train_df['NiMn_to_CrMoV'] = (train_df['Ni, wt.%'] + train_df['Mn, wt.%']) / (train_df['Cr, wt.%'] + train_df['Mo, wt.%'] + train_df['V, wt.%'] + eps)
    train_df['Ni_to_Fe'] = train_df['Ni, wt.%'] / (train_df['Fe, wt.%'] + eps)
    train_df['Cr_to_Fe'] = train_df['Cr, wt.%'] / (train_df['Fe, wt.%'] + eps)
    train_df['Mo_to_Fe'] = train_df['Mo, wt.%'] / (train_df['Fe, wt.%'] + eps)

    # 2. Geometry & Stress Features
    train_df['Volume'] = train_df['Length (mm)'] * train_df['Width (mm)'] * train_df['Thickness (mm)']
    train_df['Surface_Area'] = 2 * (
        train_df['Length (mm)'] * train_df['Width (mm)'] +
        train_df['Length (mm)'] * train_df['Thickness (mm)'] +
        train_df['Width (mm)'] * train_df['Thickness (mm)']
    )
    train_df['Compactness'] = train_df['Volume'] / (train_df['Surface_Area'] + eps)

    # Compute missing 'Area' as cross-sectional area
    train_df['Area'] = train_df['Width (mm)'] * train_df['Thickness (mm)']
    train_df['Impact_energy_per_Area'] = train_df['Impact energy (J)'] / (train_df['Area'] + eps)
    train_df['Impact_energy_per_Volume'] = train_df['Impact energy (J)'] / (train_df['Volume'] + eps)

    # 3. Elemental diversity
    element_cols = [col for col in train_df.columns if ', wt.%' in col]
    norm_composition = train_df[element_cols].div(train_df[element_cols].sum(axis=1) + eps, axis=0)
    train_df['Elemental_count'] = (train_df[element_cols] > 0).sum(axis=1)
    train_df['Std_dev_wt'] = train_df[element_cols].std(axis=1)
    train_df['Shannon_entropy_composition'] = -(norm_composition * np.log(norm_composition + eps)).sum(axis=1)

    # 4. Interaction terms
    train_df['C_times_Mn'] = train_df['C, wt.%'] * train_df['Mn, wt.%']
    train_df['Mn_times_Cr'] = train_df['Mn, wt.%'] * train_df['Cr, wt.%']
    train_df['Ni_times_Mo'] = train_df['Ni, wt.%'] * train_df['Mo, wt.%']

    # 5. Physics-inspired placeholders
    magpie_cols = train_df.columns[train_df.columns.str.contains('CovalentRadius') & train_df.columns.str.contains('range')]
    if len(magpie_cols):
        train_df['Solid_solution_strengthening'] = train_df[magpie_cols[0]] * train_df[element_cols].std(axis=1)
    else:
        train_df['Solid_solution_strengthening'] = np.nan

    train_df['Hardness_index'] = (
        0.5 * train_df['C, wt.%'] +
        0.3 * train_df['Mn, wt.%'] +
        0.2 * train_df['Mo, wt.%'] +
        0.1 * train_df['Ni, wt.%']
    )



    # 1. Composition Ratios
    test_df['C_Mn'] = test_df['C, wt.%'] / (test_df['Mn, wt.%'] + eps)
    test_df['Cr_Mo_V_Ni'] = test_df[['Cr, wt.%', 'Mo, wt.%', 'V, wt.%', 'Ni, wt.%']].sum(axis=1)
    test_df['NiMn_to_CrMoV'] = (test_df['Ni, wt.%'] + test_df['Mn, wt.%']) / (test_df['Cr, wt.%'] + test_df['Mo, wt.%'] + test_df['V, wt.%'] + eps)
    test_df['Ni_to_Fe'] = test_df['Ni, wt.%'] / (test_df['Fe, wt.%'] + eps)
    test_df['Cr_to_Fe'] = test_df['Cr, wt.%'] / (test_df['Fe, wt.%'] + eps)
    test_df['Mo_to_Fe'] = test_df['Mo, wt.%'] / (test_df['Fe, wt.%'] + eps)

    # 2. Geometry & Stress Features
    test_df['Volume'] = test_df['Length (mm)'] * test_df['Width (mm)'] * test_df['Thickness (mm)']
    test_df['Surface_Area'] = 2 * (
        test_df['Length (mm)'] * test_df['Width (mm)'] +
        test_df['Length (mm)'] * test_df['Thickness (mm)'] +
        test_df['Width (mm)'] * test_df['Thickness (mm)']
    )
    test_df['Compactness'] = test_df['Volume'] / (test_df['Surface_Area'] + eps)

    # Compute missing 'Area' as cross-sectional area
    test_df['Area'] = test_df['Width (mm)'] * test_df['Thickness (mm)']
    test_df['Impact_energy_per_Area'] = test_df['Impact energy (J)'] / (test_df['Area'] + eps)
    test_df['Impact_energy_per_Volume'] = test_df['Impact energy (J)'] / (test_df['Volume'] + eps)

    # 3. Elemental diversity
    element_cols = [col for col in test_df.columns if ', wt.%' in col]
    norm_composition = test_df[element_cols].div(test_df[element_cols].sum(axis=1) + eps, axis=0)
    test_df['Elemental_count'] = (test_df[element_cols] > 0).sum(axis=1)
    test_df['Std_dev_wt'] = test_df[element_cols].std(axis=1)
    test_df['Shannon_entropy_composition'] = -(norm_composition * np.log(norm_composition + eps)).sum(axis=1)

    # 4. Interaction terms
    test_df['C_times_Mn'] = test_df['C, wt.%'] * test_df['Mn, wt.%']
    test_df['Mn_times_Cr'] = test_df['Mn, wt.%'] * test_df['Cr, wt.%']
    test_df['Ni_times_Mo'] = test_df['Ni, wt.%'] * test_df['Mo, wt.%']

    # 5. Physics-inspired placeholders
    magpie_cols = test_df.columns[test_df.columns.str.contains('CovalentRadius') & test_df.columns.str.contains('range')]
    if len(magpie_cols):
        test_df['Solid_solution_strengthening'] = test_df[magpie_cols[0]] * test_df[element_cols].std(axis=1)
    else:
        test_df['Solid_solution_strengthening'] = np.nan

    test_df['Hardness_index'] = (
        0.5 * test_df['C, wt.%'] +
        0.3 * test_df['Mn, wt.%'] +
        0.2 * test_df['Mo, wt.%'] +
        0.1 * test_df['Ni, wt.%']
    )

    return train_df, test_df

In [5]:
feature_to_remove = []
feature_to_include = []
feature_to_include = ["C, wt.%",	"Si, wt.%",	"Mn, wt.%",	"P, wt.%",	"S, wt.%",	"Ni, wt.%",	"Cr, wt.%",	"Mo, wt.%",	"Al, wt.%",	"N, wt.%",	"Ti, wt.%",	"Fe, wt.%",	"Nb, wt.%",	"B, wt.%",	"Cu, wt.%",	"V, wt.%",	"Zr, wt.%",	"W, wt.%",	"Sn, wt.%",	"Co, wt.%",	"O, wt,%", "Ta, wt.%",	"As, wt%","Sb, wt.%",'MagpieData mean GSbandgap', 'MagpieData avg_dev GSbandgap', 'Test Temperature (C) ', 'MagpieData mean MeltingT', 'Solid_solution_strengthening', 'C_Mn', 'Embrittling_Elements', 'Impact_energy_per_Volume', 'MagpieData mean NdValence', 'Impact energy (J)', 'Impact_energy_per_Area', 'Compactness', 'MagpieData mean SpaceGroupNumber', 'Cr_to_Fe', 'MagpieData avg_dev NUnfilled', 'Area',  'MagpieData mean NValence', 'Surface_Area', 'Volume', 'MagpieData mean NUnfilled', 'MagpieData avg_dev NpUnfilled', 'MagpieData avg_dev NpValence', 'Aging_Sensitive', 'MagpieData avg_dev NsValence', 'MagpieData mean NpUnfilled', 'MagpieData avg_dev SpaceGroupNumber', 'MagpieData mean NsUnfilled', 'MagpieData mean NsValence', 'MagpieData avg_dev MeltingT', 'Ni_to_Fe', 'MagpieData mean NpValence', 'C_times_Mn', 'Elemental_count', 'NiMn_to_CrMoV', 'MagpieData avg_dev Column', 'MagpieData avg_dev NsUnfilled', 'MagpieData avg_dev NdUnfilled', 'MagpieData mean GSvolume_pa',  'MagpieData avg_dev MendeleevNumber', 'Cr_Mo_V_Ni', 'MagpieData mean Electronegativity', 'Mn_times_Cr', 'W/L', 'MagpieData mean CovalentRadius',  'L/T', 'Carbon_Equivalent', 'MagpieData avg_dev CovalentRadius']


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

train_df=pd.read_csv(TRAIN_CSV_PATH).select_dtypes(include="number")
test_df=pd.read_csv(TEST_CSV_PATH).select_dtypes(include="number")
train_df,test_df=featurizer(train_df,test_df)

target="DBTT"
print(f"Original shape: {train_df.shape}")



mean_val = train_df[target].mean()
std_val = train_df[target].std()
upper_bound = mean_val + 1.75 * std_val

train_df = train_df.dropna(subset=[target])

if feature_to_include:
    existing_features = [f for f in feature_to_include if f in train_df.columns]
    train_df = train_df[existing_features + [target]]

train_df = train_df[ (train_df[target] <= upper_bound)]

test_df = test_df.dropna(subset=[target])
test_df = test_df[list(train_df.columns)]
test_df = test_df[ (test_df[target] <= upper_bound)]

print(f"Filtered shape: {train_df.shape}")




Original shape: (3694, 193)
Filtered shape: (3451, 72)


In [7]:
len(test_df.columns)

72

In [8]:
cols_to_drop = ['DBTT', 'USE', "Width (mm)", "Length (mm)", "Thickness (mm)"] + list(feature_to_remove)
X=train_df.drop(columns=cols_to_drop, errors='ignore')
y=train_df[target]
X_test=test_df.drop(columns=cols_to_drop, errors='ignore')
y_test=test_df[[target]]

bins = pd.cut(y, bins=10, labels=False)  # 3 intervals, labels = 0,1,2

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)





In [9]:
# ====== Run Optuna ======

obj="etr"
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

if obj=="xgb":
    study = optuna.create_study(direction="minimize")
    study.optimize(xgb_objective, n_trials=50000,timeout=18000, show_progress_bar=True)

elif obj=="rf":
    study = optuna.create_study(direction="minimize")
    study.optimize(rf_objective, n_trials=50000,timeout=18000, show_progress_bar=True)
elif obj=="etr":
    study = optuna.create_study(direction="minimize")
    study.optimize(rf_objective, n_trials=50000,timeout=18000, show_progress_bar=True)



[I 2025-09-18 18:21:26,182] A new study created in memory with name: no-name-ebc9b390-6cf0-41df-b50d-4d94e4fb14ad


  0%|          | 0/50000 [00:00<?, ?it/s]

[I 2025-09-18 18:21:28,320] Trial 0 finished with value: 38.43617122691341 and parameters: {'n_estimators': 650, 'max_depth': 82, 'min_samples_split': 5, 'min_samples_leaf': 18, 'min_weight_fraction_leaf': 0.5, 'max_features': 'log2', 'bootstrap': True, 'max_samples': 0.95, 'oob_score': False, 'min_impurity_decrease': 0.48, 'ccp_alpha': 0.027}. Best is trial 0 with value: 38.43617122691341.
[I 2025-09-18 18:21:31,514] Trial 1 finished with value: 33.85974785696913 and parameters: {'n_estimators': 850, 'max_depth': 80, 'min_samples_split': 43, 'min_samples_leaf': 13, 'min_weight_fraction_leaf': 0.45, 'max_features': 'sqrt', 'bootstrap': True, 'max_samples': 1.0, 'oob_score': True, 'min_impurity_decrease': 0.6900000000000001, 'ccp_alpha': 0.0}. Best is trial 1 with value: 33.85974785696913.
[I 2025-09-18 18:21:33,805] Trial 2 finished with value: 29.746409916259687 and parameters: {'n_estimators': 650, 'max_depth': 81, 'min_samples_split': 22, 'min_samples_leaf': 15, 'min_weight_fraction

In [10]:
best_params = study.best_trial.params
print("Best parameters:", best_params)
print("Best validation MAE:", study.best_value)

from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Scale the data first
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)




if obj == "xgb":
    best_model = XGBRegressor(**best_params)
    best_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_val_scaled, y_val)],
        verbose=False
    )
    
elif obj == "rf":
    best_model = RandomForestRegressor(
        **{k: v for k, v in best_params.items() if v is not None},
        random_state=42, n_jobs=-1
    )
    best_model.fit(X_train_scaled, y_train)
    
elif obj == "etr":
    best_model = ExtraTreesRegressor(**best_params)
    best_model.fit(X_train_scaled, y_train)


Best parameters: {'n_estimators': 950, 'max_depth': 93, 'min_samples_split': 2, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.0, 'max_features': 0.8, 'bootstrap': True, 'max_samples': 1.0, 'oob_score': True, 'min_impurity_decrease': 0.0, 'ccp_alpha': 0.015}
Best validation MAE: 7.980986312030508


In [11]:
score_generator(X_train,y_train,best_model,"Training")
score_generator(X_val,y_val,best_model,"Validation")
score_generator(X_test,y_test,best_model,"Test")

X does not have valid feature names, but StandardScaler was fitted with feature names


Training
Mean Absolute Error (MAE): 3.3247
Median Absolute Percentage Error (MAPE): 0.0452
R2 Score: 0.9857
Validation
Mean Absolute Error (MAE): 7.9456
Median Absolute Percentage Error (MAPE): 0.1024
R2 Score: 0.9007


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names


Test
Mean Absolute Error (MAE): 8.4587
Median Absolute Percentage Error (MAPE): 0.1046
R2 Score: 0.8904
